## CHEME 6880 Homework 2 
Net ID : AF626

### Problem 1 
Given the following data set on whether to approve credit card applications based on the
information of four features (student status, credit rating, available credit, and age group). Please
build a Decision Tree Classifier using Information Gain to choose the best attribute, and then use
the decision tree to predict the approval outcome of a new application from an unemployed, senior
applicant with excellent credit rating and high available credit. Please provide sufficient details on
your calculation process of building the classifier, step-by-step.

In [1]:
import pandas as pd
import numpy as np

# Read problem set 1 data
data_1 = pd.read_csv('data/data_1-3.csv')
data_1

,Employment Status,Credit Rating,Available Credit,Age,Approve Application ?
0,Unemployed,Excellent,High,Young,No
1,Unemployed,Fair,High,Young,No
2,Unemployed,Excellent,High,Middle Age,Yes
3,Unemployed,Excellent,Medium,Senior,Yes
4,Employed,Excellent,Low,Senior,Yes
5,Employed,Fair,Low,Senior,No
6,Employed,Fair,Low,Middle Age,Yes
7,Unemployed,Excellent,Medium,Young,No
8,Employed,Excellent,Low,Young,Yes
9,Employed,Fair,Medium,Young,Yes


For deciding the root node and performing the first slit we ill first calculate the 
1. total entropy of target variable
2. individual entropy of feature variables
3. information gain of feature variables
4. Choose the highest information gain as our best attribute

We will keep repeating the above steps till be split into all nodes

In [2]:
# Defining general functions
# Function to calculate entropy
def calculate_entropy(y):
    total_instances = len(y)
    yes_count = sum(y)
    no_count = total_instances - yes_count
    if yes_count == 0 or no_count == 0:
        entropy = 0
    else:
        p_yes = yes_count / total_instances
        p_no = no_count / total_instances
        entropy = - (p_yes * np.log2(p_yes) + p_no * np.log2(p_no))
    return entropy

# Function to calculate information gain
def calculate_information_gain(X, y, feature):
    entropy_before_split = calculate_entropy(y)
    values = X[feature].unique()
    total_instances = len(y)
    entropy_after_split = 0
    for value in values:
        subset_indices = X.index[X[feature] == value]
        subset_y = y[subset_indices]
        entropy_subset = calculate_entropy(subset_y)
        entropy_after_split += (len(subset_y) / total_instances) * entropy_subset
    information_gain = entropy_before_split - entropy_after_split
    return information_gain

@@@@@@@@@@@@@@@@@ SPLIT 1 : 

In [3]:
df = data_1

# Initialize an empty DataFrame to store the summary of splits
summary_df = pd.DataFrame(columns=['Feature', 'Instance', 'Yes', 'No', 'Entropy'])

# Calculate entropy of the target variable
yes_count = (df['Approve Application ?'] == 'Yes').sum()
no_count = (df['Approve Application ?'] == 'No').sum()
total_instances = len(df)
entropy_S = calculate_entropy([yes_count, no_count])

# Calculate summary DataFrame
features = df.columns[:-1]
for feature in features:
    values = df[feature].unique()
    for value in values:
        instance = value
        subset_indices = df.index[df[feature] == value]
        subset_y = (df.loc[subset_indices, 'Approve Application ?'] == 'Yes').astype(int)
        yes_count = subset_y.sum()
        no_count = len(subset_y) - yes_count
        entropy = calculate_entropy(subset_y)
        summary_df.loc[len(summary_df)] = [feature, instance, yes_count, no_count, entropy]

# Display the summary DataFrame
print(summary_df)

             Feature    Instance  Yes  No   Entropy
0  Employment Status  Unemployed    4   5  0.991076
1  Employment Status    Employed    7   3  0.881291
2      Credit Rating   Excellent    7   2  0.764205
3      Credit Rating        Fair    4   6  0.970951
4   Available Credit        High    3   2  0.970951
5   Available Credit      Medium    5   4  0.991076
6   Available Credit         Low    3   2  0.970951
7                Age       Young    2   4  0.918296
8                Age  Middle Age    6   0  0.000000
9                Age      Senior    3   4  0.985228


C:\Users\af\AppData\Local\Temp\ipykernel_10504\1949047588.py:12: RuntimeWarning: invalid value encountered in log2
  entropy = - (p_yes * np.log2(p_yes) + p_no * np.log2(p_no))


In [4]:
# Calculate information gain DataFrame
information_gain_df = pd.DataFrame(columns=['Feature', 'Information Gain'])
for feature in features:
    information_gain = calculate_information_gain(df, (df['Approve Application ?'] == 'Yes').astype(int), feature)
    information_gain_df.loc[len(information_gain_df)] = [feature, information_gain]

# Display the information gain DataFrame
print(information_gain_df)

             Feature  Information Gain
0  Employment Status          0.048646
1      Credit Rating          0.108923
2   Available Credit          0.001457
3                Age          0.328974


As we can see from the information gain we got Age with the higest information gain, therefore Age feature will be our root node for the tree.<br>
Also we can see that the Middle Age candidates have all their applications approved<br>
So our tree will look like this<br><br>
Age<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Young<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Middle<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Senior<br>

@@@@@@@@@@@@@@@@@ SPLIT 2 : Going into the branch of where Age is Young

In [5]:
df = data_1[data_1['Age'] == 'Young']

# Initialize an empty DataFrame to store the summary of splits
summary_df = pd.DataFrame(columns=['Feature', 'Instance', 'Yes', 'No', 'Entropy'])

# Calculate entropy of the target variable
yes_count = (df['Approve Application ?'] == 'Yes').sum()
no_count = (df['Approve Application ?'] == 'No').sum()
total_instances = len(df)
entropy_S = calculate_entropy([yes_count, no_count])

# Calculate summary DataFrame
features = df.columns[:-1]
for feature in features:
    values = df[feature].unique()
    for value in values:
        instance = value
        subset_indices = df.index[df[feature] == value]
        subset_y = (df.loc[subset_indices, 'Approve Application ?'] == 'Yes').astype(int)
        yes_count = subset_y.sum()
        no_count = len(subset_y) - yes_count
        entropy = calculate_entropy(subset_y)
        summary_df.loc[len(summary_df)] = [feature, instance, yes_count, no_count, entropy]

# Display the summary DataFrame
print(summary_df)

             Feature    Instance  Yes  No   Entropy
0  Employment Status  Unemployed    0   4  0.000000
1  Employment Status    Employed    2   0  0.000000
2      Credit Rating   Excellent    1   2  0.918296
3      Credit Rating        Fair    1   2  0.918296
4   Available Credit        High    0   2  0.000000
5   Available Credit      Medium    1   1  1.000000
6   Available Credit         Low    1   1  1.000000
7                Age       Young    2   4  0.918296


C:\Users\af\AppData\Local\Temp\ipykernel_10504\1949047588.py:12: RuntimeWarning: invalid value encountered in log2
  entropy = - (p_yes * np.log2(p_yes) + p_no * np.log2(p_no))


In [6]:
# Calculate information gain DataFrame
information_gain_df = pd.DataFrame(columns=['Feature', 'Information Gain'])
for feature in features:
    information_gain = calculate_information_gain(df, (df['Approve Application ?'] == 'Yes').astype(int), feature)
    information_gain_df.loc[len(information_gain_df)] = [feature, information_gain]

# Display the information gain DataFrame
print(information_gain_df)

             Feature  Information Gain
0  Employment Status          0.918296
1      Credit Rating          0.000000
2   Available Credit          0.251629
3                Age          0.000000


As we can see from the information gain we got Employment Status with the higest information gain, therefore Age:Young feature will be split into Employment Status instances i.e. Unemployed and Employed.<br>
So our tree will look like this<br><br>
Age<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Young<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Employed<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Unemployed<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? NO<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Middle<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Senior<br>

@@@@@@@@@@@@@@@@@ SPLIT 3 : Going into the branch of where Age is Senior

In [7]:
df = data_1[data_1['Age'] == 'Senior']

# Initialize an empty DataFrame to store the summary of splits
summary_df = pd.DataFrame(columns=['Feature', 'Instance', 'Yes', 'No', 'Entropy'])

# Calculate entropy of the target variable
yes_count = (df['Approve Application ?'] == 'Yes').sum()
no_count = (df['Approve Application ?'] == 'No').sum()
total_instances = len(df)
entropy_S = calculate_entropy([yes_count, no_count])

# Calculate summary DataFrame
features = df.columns[:-1]
for feature in features:
    values = df[feature].unique()
    for value in values:
        instance = value
        subset_indices = df.index[df[feature] == value]
        subset_y = (df.loc[subset_indices, 'Approve Application ?'] == 'Yes').astype(int)
        yes_count = subset_y.sum()
        no_count = len(subset_y) - yes_count
        entropy = calculate_entropy(subset_y)
        summary_df.loc[len(summary_df)] = [feature, instance, yes_count, no_count, entropy]

# Display the summary DataFrame
print(summary_df)

             Feature    Instance  Yes  No   Entropy
0  Employment Status  Unemployed    1   1  1.000000
1  Employment Status    Employed    2   3  0.970951
2      Credit Rating   Excellent    3   0  0.000000
3      Credit Rating        Fair    0   4  0.000000
4   Available Credit      Medium    1   3  0.811278
5   Available Credit         Low    1   1  1.000000
6   Available Credit        High    1   0  0.000000
7                Age      Senior    3   4  0.985228


C:\Users\af\AppData\Local\Temp\ipykernel_10504\1949047588.py:12: RuntimeWarning: invalid value encountered in log2
  entropy = - (p_yes * np.log2(p_yes) + p_no * np.log2(p_no))


In [8]:
# Calculate information gain DataFrame
information_gain_df = pd.DataFrame(columns=['Feature', 'Information Gain'])
for feature in features:
    information_gain = calculate_information_gain(df, (df['Approve Application ?'] == 'Yes').astype(int), feature)
    information_gain_df.loc[len(information_gain_df)] = [feature, information_gain]

# Display the information gain DataFrame
print(information_gain_df)

             Feature  Information Gain
0  Employment Status          0.005978
1      Credit Rating          0.985228
2   Available Credit          0.235926
3                Age          0.000000


As we can see from the information gain we got Credit Rating with the higest information gain, therefore Age:Senior feature will be split into Credit Rating instances i.e. Excellent and Fair.<br>
So our tree will look like this<br><br>
Age<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Age : Young<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Employment Status : Employed<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? : YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Employment Status : Unemployed<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? : NO<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Age : Middle<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? : YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;|_ Age : Senior<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Credit Rating : Excellent<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? : YES<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Credit Rating : Fair<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|_ Application Approved ? : NO<br>

Therefore for our test case where we want to determine if a senior (unemployed) with excellent credit rating and high available credit, we can see that it will be approved as per the above decision tree

### Problem 2
Build a Naïve Bayes classifier using the training data in Problem 1, and make prediction for the
approval outcome of the same new application from an unemployed, senior applicant with
excellent credit rating and high available credit. Please provide sufficient details on your
calculation process of building the classifier, step-by-step. 

##### Solution

In [9]:
# Import required libraries
import pandas as pd

In [10]:
# Import the required data as dataframe
data_2 = pd.DataFrame(pd.read_csv('data/data_1-3.csv'))
data_2

,Employment Status,Credit Rating,Available Credit,Age,Approve Application ?
0,Unemployed,Excellent,High,Young,No
1,Unemployed,Fair,High,Young,No
2,Unemployed,Excellent,High,Middle Age,Yes
3,Unemployed,Excellent,Medium,Senior,Yes
4,Employed,Excellent,Low,Senior,Yes
5,Employed,Fair,Low,Senior,No
6,Employed,Fair,Low,Middle Age,Yes
7,Unemployed,Excellent,Medium,Young,No
8,Employed,Excellent,Low,Young,Yes
9,Employed,Fair,Medium,Young,Yes


In [11]:
# Next we calculate the overall prior probabilites
total_instances = len(data_2)
approve_yes_count = data_2[data_2['Approve Application ?'] == 'Yes'].shape[0]
approve_no_count = total_instances - approve_yes_count

p_approve_yes = approve_yes_count / total_instances
p_approve_no = approve_no_count / total_instances

print("Prior Probability of Approve = Yes:", p_approve_yes)
print("Prior Probability of Approve = No:", p_approve_no)


Prior Probability of Approve = Yes: 0.5789473684210527
Prior Probability of Approve = No: 0.42105263157894735


In [12]:
# Next we will calculate all the conditional probabilites for each combination
# Define a function to calculate conditional probabilities
def calculate_conditional_probability(feature, value, approve_value):
    count_feature_and_approve_value = data_2[(data_2[feature] == value) & (data_2['Approve Application ?'] == approve_value)].shape[0]
    count_approve_value = data_2[data_2['Approve Application ?'] == approve_value].shape[0]
    return count_feature_and_approve_value / count_approve_value

# Calculate conditional probabilities for each feature
conditional_probs = {}

for feature in data_2.columns[:-1]:
    for value in data_2[feature].unique():
        for approve_value in data_2['Approve Application ?'].unique():
            key = f"{feature} = {value} | Approve = {approve_value}"
            conditional_probs[key] = calculate_conditional_probability(feature, value, approve_value)

# Display conditional probabilities
for key, value in conditional_probs.items():
    print(key, ":", value)

Employment Status = Unemployed | Approve = No : 0.625
Employment Status = Unemployed | Approve = Yes : 0.36363636363636365
Employment Status = Employed | Approve = No : 0.375
Employment Status = Employed | Approve = Yes : 0.6363636363636364
Credit Rating = Excellent | Approve = No : 0.25
Credit Rating = Excellent | Approve = Yes : 0.6363636363636364
Credit Rating = Fair | Approve = No : 0.75
Credit Rating = Fair | Approve = Yes : 0.36363636363636365
Available Credit = High | Approve = No : 0.25
Available Credit = High | Approve = Yes : 0.2727272727272727
Available Credit = Medium | Approve = No : 0.5
Available Credit = Medium | Approve = Yes : 0.45454545454545453
Available Credit = Low | Approve = No : 0.25
Available Credit = Low | Approve = Yes : 0.2727272727272727
Age = Young | Approve = No : 0.5
Age = Young | Approve = Yes : 0.18181818181818182
Age = Middle Age | Approve = No : 0.0
Age = Middle Age | Approve = Yes : 0.5454545454545454
Age = Senior | Approve = No : 0.5
Age = Senior |

Now that we have the prior probabilites and the conditional probabilities for each case we can go ahead with the prediction for our new case

In [13]:
# Define a function to make predictions
def predict_approval(application):
    # Calculate probability for Approve = Yes
    p_approve_yes_given_application = p_approve_yes
    for feature, value in application.items():
        key = f"{feature} = {value} | Approve = Yes"
        if key in conditional_probs:
            p_approve_yes_given_application = p_approve_yes_given_application * conditional_probs[key]

    # Calculate probability for Approve = No
    p_approve_no_given_application = p_approve_no
    for feature, value in application.items():
        key = f"{feature} = {value} | Approve = No"
        if key in conditional_probs:
            p_approve_no_given_application = p_approve_no_given_application * conditional_probs[key]

    # Normalize probabilities
    total_probability = p_approve_yes_given_application + p_approve_no_given_application
    p_approve_yes_given_application = p_approve_yes_given_application / total_probability
    p_approve_no_given_application = p_approve_no_given_application / total_probability

    return {"Approve = Yes": p_approve_yes_given_application, "Approve = No": p_approve_no_given_application}

# New application data
new_application = {
    'Employment Status': 'Unemployed',
    'Credit Rating': 'Excellent',
    'Available Credit': 'High',
    'Age': 'Senior'
}

# Make prediction
prediction = predict_approval(new_application)
print("Prediction:", prediction)


Prediction: {'Approve = Yes': 0.5478633059311094, 'Approve = No': 0.45213669406889057}


From the above we can see that for an unemployed, senior applicant with excellent credit rating and high available credit, as per Naïve Bayes classifier, there is around 54 % chance that the application will get approved but there is a 45 % chance that the application might get rejected

### Problem 3
Build k-nearest neighbor classifiers for the cases of k=1, k=3, and k=5, using the training data in
Problem 1, by converting all features of the data instances into numerical values. You can use any
distance function that you prefer, but please define it explicitly. After building these three
classifiers, please use them to make prediction for the approval outcome of the same new
application from an unemployed, senior applicant with excellent credit rating and high available
credit. Please provide sufficient details of your calculation of building the k-nearest neighbors
classifiers, step-by-step. Please use the following table to convert all features of the instances into
numerical values

##### Solution

In [14]:
# Import required libararies
import pandas as pd
import numpy as np

In [15]:
# Import the required data as dataframe
data_3 = pd.DataFrame(pd.read_csv('data/data_1-3.csv'))
data_3

,Employment Status,Credit Rating,Available Credit,Age,Approve Application ?
0,Unemployed,Excellent,High,Young,No
1,Unemployed,Fair,High,Young,No
2,Unemployed,Excellent,High,Middle Age,Yes
3,Unemployed,Excellent,Medium,Senior,Yes
4,Employed,Excellent,Low,Senior,Yes
5,Employed,Fair,Low,Senior,No
6,Employed,Fair,Low,Middle Age,Yes
7,Unemployed,Excellent,Medium,Young,No
8,Employed,Excellent,Low,Young,Yes
9,Employed,Fair,Medium,Young,Yes


In [16]:
# Mapping categorical features to numerical values
numerical_data_mapping = {
    'Employment Status': {'Unemployed': 0, 'Employed': 1},
    'Credit Rating': {'Excellent': 1, 'Fair': 0},
    'Available Credit': {'High': 2, 'Medium': 1, 'Low': 0},
    'Age': {'Senior': 2, 'Middle Age': 1, 'Young': 0}
}

# Apply mapping to the DataFrame
data_3_numerical = data_3.copy()
for col, mapping in numerical_data_mapping.items():
    data_3_numerical[col] = data_3[col].map(mapping)
data_3_numerical

,Employment Status,Credit Rating,Available Credit,Age,Approve Application ?
0,0,1,2,0,No
1,0,0,2,0,No
2,0,1,2,1,Yes
3,0,1,1,2,Yes
4,1,1,0,2,Yes
5,1,0,0,2,No
6,1,0,0,1,Yes
7,0,1,1,0,No
8,1,1,0,0,Yes
9,1,0,1,0,Yes


In [17]:
# Next building the KNN Classifier
# For disctance calculation I am using Euclidean distance
# Defining distance function (Euclidean distance)
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# Define the k-nearest neighbors algorithm
def k_nearest_neighbors(train_data, test_instance, k):
    distances = []
    for index, row in train_data.iterrows():
        dist = euclidean_distance(row[:-1], test_instance)
        distances.append((index, dist))

    distances.sort(key=lambda x: x[1])
    neighbors = [train_data.iloc[index] for index, _ in distances[:k]]
    return neighbors

# Define a function to predict the class of a test instance using k-nearest neighbors
def predict_class(neighbors):
    votes = [neighbor[-1] for neighbor in neighbors]
    return max(set(votes), key=votes.count)

In [18]:
# Convert the new application instance to numerical values
new_application = [0, 1, 2, 2]  # Unemployed, Excellent, High, Senior

# Build k-nearest neighbor classifiers for k=1, k=3, and k=5
for k in [1, 3, 5]:
    neighbors = k_nearest_neighbors(data_3_numerical, new_application, k)
    prediction = predict_class(neighbors)
    print(f"For k={k}, predicted approval outcome: {prediction}")

For k=1, predicted approval outcome: Yes
For k=3, predicted approval outcome: Yes
For k=5, predicted approval outcome: Yes


C:\Users\af\AppData\Local\Temp\ipykernel_10504\1493716506.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  votes = [neighbor[-1] for neighbor in neighbors]


### Problem 4
Use a software package to re-solve the classification problems above. Please use the original data
set, and build the following classifiers to predict the approval outcome of a new application from
an unemployed, senior applicant with excellent credit rating and high available credit.
(a) Decision tree.
(b) Naïve Bayes.
(c) k-nearest neighbor for the cases of k=1, k=3, and k=5.
(d) Support vector machine.

##### Solution

In [19]:
# Importing the required libraries
import pandas as pd

# Getting data for problem 4
data_4 = pd.read_csv('data/data_1-3.csv')
data_4

,Employment Status,Credit Rating,Available Credit,Age,Approve Application ?
0,Unemployed,Excellent,High,Young,No
1,Unemployed,Fair,High,Young,No
2,Unemployed,Excellent,High,Middle Age,Yes
3,Unemployed,Excellent,Medium,Senior,Yes
4,Employed,Excellent,Low,Senior,Yes
5,Employed,Fair,Low,Senior,No
6,Employed,Fair,Low,Middle Age,Yes
7,Unemployed,Excellent,Medium,Young,No
8,Employed,Excellent,Low,Young,Yes
9,Employed,Fair,Medium,Young,Yes


In [20]:
# Mapping for categorical variables
mapping = {
    "Employment Status": {"Unemployed": 0, "Employed": 1},
    "Credit Rating": {"Excellent": 1, "Fair": 0},
    "Available Credit": {"High": 2, "Medium": 1, "Low": 0},
    "Age": {"Senior": 2, "Middle Age": 1, "Young": 0},
    "Approve Application": {"No": 0, "Yes": 1}
}

temp_data_4 = data_4.copy()
# Apply mapping to the dataframe
temp_data_4.replace(mapping, inplace=True)

# Separating features and target variable
x = temp_data_4.drop(columns=["Approve Application ?"])
y = temp_data_4["Approve Application ?"]

In [21]:
x

,Employment Status,Credit Rating,Available Credit,Age
0,0,1,2,0
1,0,0,2,0
2,0,1,2,1
3,0,1,1,2
4,1,1,0,2
5,1,0,0,2
6,1,0,0,1
7,0,1,1,0
8,1,1,0,0
9,1,0,1,0


In [22]:
y

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12     No
13     No
14    Yes
15     No
16    Yes
17     No
18    Yes
Name: Approve Application ?, dtype: object

In [23]:
# Make prediction for new application
new_application = [[0, 1, 2, 2]]  # Unemployed, Excellent Credit Rating, High Available Credit, Senior

Now, that we have transformed the features and target variable we will go ahead with building the different classifiers and predict for new case

a) Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier

# Train Decision Tree classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x, y)
prediction_decision_tree = decision_tree.predict(new_application)
print("Decision Tree prediction:", prediction_decision_tree)

Decision Tree prediction: ['Yes']


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


b) Naïve Baye

In [25]:
from sklearn.naive_bayes import GaussianNB

# Train Naïve Bayes classifier
naive_bayes = GaussianNB()
naive_bayes.fit(x, y)

# Make prediction for new application
prediction_naive_bayes = naive_bayes.predict(new_application)
print("Naïve Bayes prediction:", prediction_naive_bayes)

Naïve Bayes prediction: ['Yes']


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


(c) k-nearest neighbor for k=1, k=3, and k=5:

In [26]:
from sklearn.neighbors import KNeighborsClassifier

# Train KNN classifiers for k=1, k=3, and k=5
for k in [1, 3, 5]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x, y)

    # Make prediction for new application
    prediction_knn = knn.predict(new_application)
    print("KNN (k={}) prediction:".format(k), prediction_knn)

KNN (k=1) prediction: ['Yes']
KNN (k=3) prediction: ['Yes']
KNN (k=5) prediction: ['Yes']


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


(d) Support Vector Machine

In [27]:
from sklearn.svm import SVC

# Train Support Vector Machine classifier
svm = SVC(kernel='linear')
svm.fit(x, y)

# Make prediction for new application
prediction_svm = svm.predict(new_application)
print("SVM prediction:", prediction_svm)

SVM prediction: ['Yes']


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Summarizing the output from all calssifiers we get:

In [28]:
print("Decision Tree prediction:", prediction_decision_tree)
print("Naïve Bayes prediction:", prediction_naive_bayes)
for k in [1, 3, 5]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x, y)

    # Make prediction for new application
    prediction_knn = knn.predict(new_application)
    print("KNN (k={}) prediction:".format(k), prediction_knn)
print("SVM prediction:", prediction_svm)

Decision Tree prediction: ['Yes']
Naïve Bayes prediction: ['Yes']
KNN (k=1) prediction: ['Yes']
KNN (k=3) prediction: ['Yes']
KNN (k=5) prediction: ['Yes']
SVM prediction: ['Yes']


c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


### Problem 5
Develop two Support Vector Machine classifiers for the following training data set (16 data
examples) using two approaches:
(a) Choose a suitable polynomial kernel to build the SVM classifier;
(b) Use Gaussian kernel (set standard deviation δ to 1) to develop the SVM classifier.
Please provide the functional form of the decision boundary for subproblem (a), and present the
predictions of both classifiers to predict the outcome for [1.5 1.5 1.5]. Please explicitly present the
optimization proble

##### Solution

In [45]:
# Preparing the data for the problem
data_5 = pd.read_csv('data/data_5-6.csv')
data_5

,X1,X2,X3,Y
0,1,1,1,Yes
1,1,1,-1,Yes
2,1,-1,1,Yes
3,-1,1,1,Yes
4,-1,-1,1,Yes
5,-1,1,-1,Yes
6,1,-1,-1,Yes
7,-1,-1,-1,Yes
8,2,2,-2,No
9,2,-2,2,No


In [46]:
# Converting the data on the into the required format
data_5['Y'] = data_5['Y'].map({'Yes' : 1, 'No' : -1}) # Mapping yes or no to numeric values
data_5

,X1,X2,X3,Y
0,1,1,1,1
1,1,1,-1,1
2,1,-1,1,1
3,-1,1,1,1
4,-1,-1,1,1
5,-1,1,-1,1
6,1,-1,-1,1
7,-1,-1,-1,1
8,2,2,-2,-1
9,2,-2,2,-1


In [47]:
#Splitting data into x and y
feature = data_5[['X1','X2','X3']]
outcome = data_5[['Y']]

a) Now for the Polynomial Kernel :<br>
The kernel equation can be given as : <br>
$K(x,x_{i}) = (x⋅x_{i} + 1)^{d}$
<br>
Where:
x = feature 
xi = outcome
d = d is the degree of the polynomial kernel.
<br>
Here I am considering a 2 order polynomial kernel, so we will have the kernel equation as :<br>
$K(x,x_{i}) = (x⋅x_{i} + 1)^{2}$
<br>
Now, the functional form of the decision boundary is given as :
$$ f(\mathbf{x}) = \text{sign} \left( \sum_{i=1}^{N_{SV}} \alpha_i y_i (\mathbf{x} \cdot \mathbf{x}_i + 1)^2 + b \right) $$

\begin{align*}
& f(\mathbf{x}) \text{ represents the decision boundary function.} \\
& \mathbf{x} \text{ is the input feature vector.} \\
& N_{SV} \text{ is the number of support vectors.} \\
& \alpha_i \text{ are the Lagrange multipliers corresponding to the support vectors.} \\
& y_i \text{ are the class labels of the support vectors.} \\
& \mathbf{x}_i \text{ represents the } i\text{-th support vector.} \\
& 2 \text{ is the degree of the polynomial kernel, in our case 2} \\
& b \text{ is the bias term.}
\end{align*}



b) Now for the Gaussian Kernel, we have the kernel equation as:<br>
$$
K(\mathbf{x}, \mathbf{x}_i) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}_i\|^2}{2\sigma^2}\right)
$$
<br>
And the decision boundary condition is given as : <br>

$$
f(\mathbf{x}) = \text{sign} \left( \sum_{i=1}^{N_{SV}} \alpha_i y_i \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}_i\|^2}{2\sigma^2}\right) + b \right)
$$


b) Now for the Gausian Kernel, the kernel function can be given as:
$$
K(\mathbf{x}, \mathbf{x}_i) = \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}_i\|^2}{2\sigma^2}\right)
$$

- $K(\mathbf{x}, \mathbf{x}_i):$ Gaussian kernel function between feature vectors \( \mathbf{x} \) and \( \mathbf{x}_i \).
- $ \|\mathbf{x} - \mathbf{x}_i\| $ Euclidean distance between feature vectors $ \mathbf{x} $ and $ \mathbf{x}_i $.
- $ \sigma $ Bandwidth parameter of the Gaussian kernel, controlling the width of the kernel.

And the Decision boundary can be given as:
$$
f(\mathbf{x}) = \text{sign} \left( \sum_{i=1}^{N_{SV}} \alpha_i y_i \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}_i\|^2}{2\sigma^2}\right) + b \right)
$$

- $ f(\mathbf{x}) $ Decision boundary function.
- $ N_{SV} $ Number of support vectors.
- $ \alpha_i $ Lagrange multipliers corresponding to the support vectors.
- $ y_i $ Class labels of the support vectors.
- $ \mathbf{x}_i $ $ i $-th support vector.
- $ \sigma $ Bandwidth parameter of the Gaussian kernel.
- $ b $ Bias term.


### Problem 6
Use a software package to re-solve Problem 5 (classification via SVM). Please use the same data
set and build the following classifiers to predict the outcome for [1.5 1.5 1.5].
(a) Support vector machine with a polynomial kernel.
(b) Support vector machine with a Gaussian kernel.

##### Solution

In [29]:
# Import the required libararies
import pandas as pd
from sklearn.svm import SVC

# Load the dataset
data_6 = pd.read_csv('data/data_5-6.csv')

# Separate features and target variable
x = df.drop(columns=["Y"])
y = df["Y"]

# Define new data point
new_point = [[1.5, 1.5, 1.5]]

# SVM with polynomial kernel
svm_poly = SVC(kernel='poly')
svm_poly.fit(x, y)
prediction_poly = svm_poly.predict(new_point)
print("SVM with Polynomial kernel prediction:", prediction_poly[0])

# SVM with Gaussian kernel
svm_gaussian = SVC(kernel='rbf')
svm_gaussian.fit(x, y)
prediction_gaussian = svm_gaussian.predict(new_point)
print("SVM with Gaussian kernel prediction:", prediction_gaussian[0])

KeyError: "['Y'] not found in axis"

From the above results we can see that for the SVM classifier using Polynomial Kernel gave a predicition of Yes whereas for the Gaussian Kernel it predicted as No, indicating that importance of the choice of kernel. Polynomial kernel is better in capturing the complex linear boundry conditions whereas Gaussian is better at capturing the non-linear decision boundaries